In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab/ai_scholar/'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
import sys
import pandas as pd
sys.path.append(GOOGLE_DRIVE_PATH)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install efficiency

In [ ]:
from get_strong_associate_firstname import hello_test, cal_PUB_per, read_USSA_data, classify_gender, count_classified_authors, Name2Gender

In [ ]:
hello_test()

In [ ]:
male_associate_, female_associate_ = cal_PUB_per()
male_associate, female_associate = read_USSA_data()
male_names = male_associate_ + male_associate
female_names = female_associate + female_associate_
unique_male_names = set(male_names)
unique_female_names = set(female_names)

In [ ]:
unique_male_names.intersection(unique_female_names)

In [ ]:
classify_gender(unique_male_names, unique_female_names, 'author_gender_new.csv')

In [ ]:
count_classified_authors('../../author_gender.csv'), count_classified_authors('author_gender_new.csv')

In [ ]:
name2gender = Name2Gender()

In [ ]:
df = pd.read_csv(GOOGLE_DRIVE_PATH+'data/gender_preprocess/author_gender_new.csv', names=('name', 'url', 'gender'))
# df.insert(0, 'index', df.index)
for index, row in df.iterrows():
  row['gender'] = name2gender.lookup_gender(row['name'])
df.to_csv(GOOGLE_DRIVE_PATH+'data/gender_preprocess/author_gender_new_update.csv', index=False)

In [ ]:
def classify_gender_by_name(male_asso_, female_asso_, male_asso, female_asso, fullname):
  gender = name2gender.full_name2gender.get(fullname.lower(), '-')
  if gender == 'M':
    print(f'{fullname}: predicted M in ACL-male')
  elif gender == 'F':
    print(f'{fullname}: predicted F in ACL-female')
  else:
    firstname = fullname.split(' ')[0].lower()
    if firstname in set(male_asso_):
      print(f'{fullname}: predicted M in PUBMED')
    elif firstname in set(female_asso_):
      print(f'{fullname}: predicted F in PUBMED')
    if firstname in set(male_asso):
      print(f'{fullname}: predicted M in USSA')
    elif firstname in set(female_asso):
      print(f'{fullname}: predicted F in USSA')
classify_gender_by_name(male_associate_, female_associate_, male_associate, female_associate, 'mushtaq ')

In [ ]:
import numpy as np
scholar_all = np.load(GOOGLE_DRIVE_PATH+'78k_scholar_new.npy', allow_pickle=True)

In [ ]:
hand_label_dict = {}
with open(GOOGLE_DRIVE_PATH+'data/gender_preprocess/hand_labels.txt', 'r') as infile:
  for line in infile.readlines():
    line = line.strip()
    line1, line2 = line.rsplit(', ', 1)
    hand_label_dict[line1] = line2

In [ ]:
df = pd.read_csv(GOOGLE_DRIVE_PATH+'data/gender_preprocess/author_gender_new_update.csv')
cites_all = np.array([int(scholar['citations_table'][0]) for scholar in scholar_all])
sorted_cites_idx = cites_all.argsort()[::-1]
cnt = 0
for idx in sorted_cites_idx:
  if cnt >= 1000:
    break
  name = scholar_all[idx]['name']
  if name in hand_label_dict.keys():
    df.loc[df['url'] == scholar_all[idx]['url'], 'gender'] = hand_label_dict[name]
  cnt += 1
df.to_csv(GOOGLE_DRIVE_PATH+'data/gender_preprocess/author_gender_new_update1.csv', index=False)

In [ ]:
df = pd.read_csv(GOOGLE_DRIVE_PATH+'data/gender_preprocess/author_gender_new_update.csv')
with open(GOOGLE_DRIVE_PATH+'data/gender_preprocess/hand_labels.txt', 'r') as infile:
  for line in infile.readlines():
    line = line.strip()
    line1, line2 = line.rsplit(', ', 1)
    df.loc[df['name'] == line1, 'gender'] = line2
    # print(df.loc[df['name'] == line1, 'gender'])
df.to_csv(GOOGLE_DRIVE_PATH+'data/gender_preprocess/author_gender_new_update1.csv', index=False)